# Deep Learning Models etc

In [3]:
from IPython.display import HTML
import import_ipynb
import random
import utils

importing Jupyter notebook from utils.ipynb


In [4]:
import numpy as np
import torch
from torch import nn
from torch import optim
from IPython import display

Compute accuracy of predictions.

In [4]:
def accuracy(Net,X_test,y_test,verbose=True):
    Net.eval()
    m = X_test.shape[0]
    y_pred = Net(X_test)
    predicted = torch.max(y_pred, 1)[1]
    correct = (predicted == y_test).float().sum().item()
    if verbose: print(correct,m)
    accuracy = correct/m
    Net.train()
    return accuracy

Generic training loop

In [ ]:
def Train(Net,data,epochs=20,lr=None,Loss=nn.NLLLoss(),verbose=False):
    if lr!=None: Net.optimizer = optim.Adam(Net.parameters(),lr=lr)
    losses = []
    accs = []
    for e in range(epochs):
        step=0
        tot_loss=0.0
        acc=0.0
        b=0
        for (X,y) in data:
            y_pred = Net(X)
            loss = Loss(y_pred,y)
            Net.optimizer.zero_grad()
            loss.backward()
            Net.optimizer.step()
            step+=1
            tot_loss+=loss
            acc+=accuracy(Net,X,y,verbose=False)
            b+=1
        l = tot_loss.item()/step
        a = acc/step
        losses += [l]
        accs += [a]
        if verbose: 
            print('Epoch  % 2d Loss: %2.5e Accuracy: %2.5f'%(e,l,a))
            display.clear_output(wait=True)
    return Net,losses,accs

Multi-layer perceptron with ReLU non-lineartities; for classification or regression.

In [7]:
class MLP(nn.Module):
    def __init__(self,dims=[5,3,2],task='classification',lr=1e-3):
        super(MLP,self).__init__()
        self.dims=dims
        self.n = len(self.dims)-1
        self.task=task
        self.layers=nn.ModuleList()
        for i in range(self.n-1):
            self.layers.append(nn.Linear(dims[i],dims[i+1]))
            self.layers.append(nn.ReLU())
        if task=='classification': 
            self.layers.append(nn.Linear(dims[i+1],dims[i+2]))
            self.layers.append(nn.LogSoftmax(dim=1))
        elif task=='regression': 
            self.layers.append(nn.Linear(dims[i+1],dims[i+2]))
            self.layers.append(nn.Linear(dims[i+2],1))
        else: self.layers.append(nn.Linear(dims[i+1],dims[i+2]))
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
    def forward(self,x):
        for l in self.layers:
            x = l(x)
        return(x)

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,lr):
        # This just calls the base class constructor
        super().__init__()
        self.input_size=input_size
        # Neural network layers assigned as attributes of a Module subclass
        # have their parameters registered for training automatically.
        self.rnn = nn.RNN(input_size, hidden_size, nonlinearity='relu', batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)
        self.logsoft = nn.LogSoftmax(dim=-1)
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
    def forward(self, x):
        # The RNN also returns its hidden state but we don't use it.
        # While the RNN can also take a hidden state as input, the RNN
        # gets passed a hidden state initialized with zeros by default.
        if self.input_size==1:x=x.unsqueeze(-1)
        h = self.rnn(x)[0]
        x = self.linear(h)
        x = self.logsoft(x)
        x = x[:,-1,:]
        return x

In [8]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_sizeL, output_size, lr):
        super().__init__()
        self.input_size=input_size
        self.lstm = torch.nn.LSTM(input_size, hidden_sizeL[0], batch_first=True)
        self.linear1 = torch.nn.Linear(hidden_sizeL[0], hidden_sizeL[1])
        self.linear2 = torch.nn.Linear(hidden_sizeL[1], output_size)
        self.logsoft = nn.LogSoftmax(dim=-1)
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
        self.dropout = nn.Dropout(.3)
        self.relu = nn.ReLU()
    def forward(self, x):
        if self.input_size==1:x=x.unsqueeze(-1)
        h = self.lstm(x)[0]
        x = h[:,-1,:]
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.logsoft(x)
        return x